In [6]:
!pip install transformers datasets bitsandbytes accelerate sentence-transformers faiss-cpu -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 14.0 MB/s eta 0:00:00


In [1]:
from huggingface_hub import login
login(token="  ")

In [2]:
import warnings

# Suppress all warnings or just that specific one
warnings.filterwarnings("ignore", category=UserWarning, module="huggingface_hub.utils._auth")

In [3]:
from transformers import pipeline

# Load a small open-access model
generator = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct", device= -1)

# Try a simple prompt
output = generator("Explain why LLMs are useful in AI systems.", max_new_tokens=100)
print(" Generated Text:\n", output[0]['generated_text'])


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


💬 Generated Text:
 Explain why LLMs are useful in AI systems. LLMs are Large Language Models, which are trained on vast amounts of text data to generate human-like language. They can be used in various applications, such as chatbots, language translation, and content creation. Here are some reasons why LLMs are useful in AI systems:

1. **Improved language understanding**: LLMs can be trained to understand the nuances of language, including idioms, colloquialisms, and context-dependent expressions. This enables them to generate more accurate and context


In [4]:
from transformers import logging
logging.set_verbosity_error()

In [5]:
from transformers import AutoTokenizer

models = {
    "phi3": "microsoft/Phi-3-mini-4k-instruct",
    "qwen2": "Qwen/Qwen2-7B-Instruct",
    "starcoder2": "bigcode/starcoder2-3b"
}

text = "Tokenization helps break down sentences into model-friendly pieces."

for name, model_id in models.items():
    print(f"\n {name.upper()} — {model_id}")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokens = tokenizer.encode(text)
    print(" Token IDs:", tokens)
    print(" Decoded:", tokenizer.batch_decode([tokens]))



 PHI3 — microsoft/Phi-3-mini-4k-instruct
 Token IDs: [25159, 2133, 6911, 2867, 1623, 25260, 964, 1904, 29899, 18326, 368, 12785, 29889]
 Decoded: ['Tokenization helps break down sentences into model-friendly pieces.']

 QWEN2 — Qwen/Qwen2-7B-Instruct
 Token IDs: [3323, 2022, 8609, 1438, 1495, 22870, 1119, 1614, 21896, 9666, 13]
 Decoded: ['Tokenization helps break down sentences into model-friendly pieces.']

 STARCODER2 — bigcode/starcoder2-3b
 Token IDs: [1781, 1668, 15783, 1644, 2850, 29031, 2012, 1573, 50, 25308, 23072, 51]
 Decoded: ['Tokenization helps break down sentences into model-friendly pieces.']


In [6]:
from sentence_transformers import SentenceTransformer, util

# 1. Create a small knowledge base
kb = [
    "LangChain is a framework to build LLM applications.",
    "LoRA is a method to fine-tune LLMs efficiently.",
    "Hugging Face hosts thousands of models and datasets.",
    "RAG stands for Retrieval-Augmented Generation."
]

# 2. Load sentence transformer to embed knowledge
retriever = SentenceTransformer('all-MiniLM-L6-v2')
kb_embeddings = retriever.encode(kb, convert_to_tensor=True)

# 3. Embed a user query
query = "What is RAG in language models?"
query_embedding = retriever.encode(query, convert_to_tensor=True)

# 4. Compute cosine similarity
scores = util.pytorch_cos_sim(query_embedding, kb_embeddings)
top_index = scores.argmax().item()
retrieved_doc = kb[top_index]

print(" Retrieved Context:", retrieved_doc)

# 5. Prompt LLM with retrieved context
prompt = f"Context: {retrieved_doc}\n\nQuestion: {query}\nAnswer:"
response = generator(prompt, max_new_tokens=60)
print("\n LLM Response:", response[0]['generated_text'])


 Retrieved Context: RAG stands for Retrieval-Augmented Generation.

 LLM Response: Context: RAG stands for Retrieval-Augmented Generation.

Question: What is RAG in language models?
Answer: Retrieval-Augmented Generation (RAG)

Explanation: RAG is a training method used in language models that combines the strengths of retrieval-based and generation-based methods. It involves training a model to retrieve a specific piece of text from a large corpus and then using the retrieved text as input to generate new


In [ ]:
#!pip install peft

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType

# Load base model
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# Apply LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],  # Specific to Falcon-like architectures
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())

print(f" Trainable parameters: {trainable:,} / {total:,}")
print("Now LoRA is actually applied.")


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
#!pip install --upgrade --force-reinstall transformers

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the instruction-tuned model and tokenizer
model_name = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

prompt = "Why is the sky blue?"

# Tokenize and move to model's device
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,        # Greedy decoding for factual response
        temperature=0.7,
        top_p=0.9
    )

# Decode and print response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Why is the sky blue? The sky appears blue to the human eye because of the way Earth's atmosphere scatters sunlight. Sunlight is made up of different colors, which are scattered in all directions by the gases and particles in the air. Blue light is scattered more than other colors because it travels as shorter, smaller waves. This is known as Rayleigh scattering. When we look up at the sky, we see the scattered blue light from every direction, which makes the sky appear blue.
